### Meta Classes and Abstract Base Classes (ABCs)

- [Customizing class creation](https://docs.python.org/2/reference/datamodel.html#customizing-class-creation)

- [Customizing instance and subclass checks](https://docs.python.org/2/reference/datamodel.html#customizing-instance-and-subclass-checks)

- [A Primer on Python Metaclasses](https://jakevdp.github.io/blog/2012/12/01/a-primer-on-python-metaclasses/)

- [What is a meta class in Python? - StackOverflow](http://stackoverflow.com/questions/100003/what-is-a-metaclass-in-python)



In [13]:
"""
In python, literally, everything is an object.
"""

class DoNothing(object):
    pass

l = [1,2,3,4]
print type(l) 
    # <type 'list'>
print type(1), type(1.1), type((1,2)), type({'a':'a','b':'b'})
    # <type 'int'> <type 'float'> <type 'tuple'> <type 'dict'>
print type(DoNothing())
    # <class '__main__.DoNothing'>
print type(DoNothing), type(int), type(float), type(tuple), type(list), type(dict), type(type)
    # <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'>
    
""" isinstance(object, classinfo) """
print isinstance(type, object), isinstance(object, type)
    # True True
    
""" isinstance(class, parent) """
print issubclass(type, object), issubclass(object, type)
    # True False

<type 'list'>
<type 'int'> <type 'float'> <type 'tuple'> <type 'dict'>
<class '__main__.DoNothing'>
<type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'>
True True
True False


In [9]:
"""
Classes as objects

    class MyClass(object): pass === MyClass = type('MyClass', (), {})
    
    type(name, bases, dct):
    
        1. name is a string giving the name of the class to be constructed
        2. bases is a tuple giving the parent classes of the class to be constructed
        3. dct is a dictionary of the attributes and methods of the class to be constructed
    
"""

def class_factory():
    class Foo(object):
        pass
    return Foo

F = class_factory()
f = F()
print(type(f)) # <class '__main__.Foo'>

def class_factory():
    return type('Foo', (), {})

F = class_factory()
f = F()
print(type(f)) # <class '__main__.Foo'>

class Foo(object):
    i = 4
    
class Bar(Foo):
    def get_i(self):
        return self.i
    
b = Bar()
print b.get_i() # 4

Foo = type('Foo', (), {'i':4})
Bar = type('Bar', (Foo,), {'get_i': lambda self: self.i})

b = Bar()
print b.get_i() # 4    


<class '__main__.Foo'>
<class '__main__.Foo'>
4
4


### Customizing class creation: 

- By default, new-style classes are constructed using ***type()***. 


- A class definition is read into a separate namespace and the value of class name is bound to the result of ***type(name, bases, dict)***.


- When the class definition is read, if **\_\_metaclass\_\_** is defined then the **callable assigned to it** will be called instead of **type()**. 


This ***allows*** classes or functions to be written which monitor or alter the class creation process:


- Modifying the class dictionary prior to the class being created.

- Returning an instance of another class – essentially performing the role of a factory function.
    
    
These steps will have to be performed in the metaclass’s ***\_\_new\_\_()*** method – 

- **type.\_\_new\_\_()** can then be called from this method to create a class with different properties. 
    

This **example** adds a new element to the class dictionary before creating the class:

```
class metacls(type):
    def __new__(mcs, name, bases, dict):
        dict['foo'] = 'metacls was here'
        return type.__new__(mcs, name, bases, dict)
```

** \_\_metaclass\_\_ **
- This variable can be any callable accepting arguments for ***name, bases, and dict***. 
- Upon class creation, the callable is used instead of the built-in **type()**.

`New in version 2.2.`

The appropriate metaclass is determined by the following precedence rules:

1. If dict[`'__metaclass__'`] exists, it is used.
2. Otherwise, if there is at least one base class, its metaclass is used (this looks for a `__class__` attribute first and if not found, uses its type).
3. Otherwise, if a global variable named `__metaclass__` exists, it is used.
4. Otherwise, the old-style, classic metaclass (types.ClassType) is used.


The potential uses for metaclasses are boundless. Some ideas that have been explored including:
- logging, 
- interface checking, 
- automatic delegation, 
- automatic property creation, 
- proxies, 
- frameworks, and 
- automatic resource locking/synchronization.

In [18]:
"""
Customizing class creation

Example 1: Modifying Attributes

    Shows how metaclasses can be used to create powerful and flexible APIs 

"""

from os.path import expanduser
import pprint

def create_a_tmp_file(file_name):
    return expanduser('~') + '/' + file_name + '.txt'

class InterfaceMeta(type):
    def __new__(cls, name, parents, dct):  
        """ here is the trick happens: type is callable and __new__ is overriden """
        # create a class_id if it's not specified
        if 'class_id' not in dct:
            dct['class_id'] = name.lower()
        
        # open the specified file for writing
        if 'file' in dct:
            filename = dct['file']
            dct['file'] = open(filename, 'w')
        
        # we need to call type.__new__ to complete the initialization
        return super(InterfaceMeta, cls).__new__(cls, name, parents, dct)
    
Interface = InterfaceMeta('Interface', (), dict(file=create_a_tmp_file('tmp')))

print(Interface.class_id)
print(Interface.file)
print type(Interface)
print

class Interface(object):
    """ InterfaceMeta is assigned to __metaclass__ """
    __metaclass__ = InterfaceMeta
    file = create_a_tmp_file('tmp')
    
print(Interface.class_id)
print(Interface.file)
print type(Interface)
print

class UserInterface(Interface):
    file = create_a_tmp_file('foo')
    
print(UserInterface.class_id)
print(UserInterface.file)
print type(UserInterface)


interface
<open file 'C:\\Users\\jizhe/tmp.txt', mode 'w' at 0x00000000039A5E40>
<class '__main__.InterfaceMeta'>

interface
<open file 'C:\\Users\\jizhe/tmp.txt', mode 'w' at 0x00000000039A5F60>
<class '__main__.InterfaceMeta'>

userinterface
<open file 'C:\\Users\\jizhe/foo.txt', mode 'w' at 0x00000000039A5ED0>
<class '__main__.InterfaceMeta'>


In [21]:
"""
Customizing class creation

Example 2: Registering Subclasses

    Shows how to automatically register all subclasses derived from a given base class.

"""

pp = pprint.PrettyPrinter(indent=2)

class DBInterfaceMeta(type):
    # we use __init__ rather than __new__ here because we want
    # to modify attributes of the class *after* they have been
    # created
    def __init__(cls, name, bases, dct):
        if not hasattr(cls, 'registry'):
            # this is the base class.  Create an empty registry
            cls.registry = {}
        else:
            # this is a derived class.  Add cls to the registry
            interface_id = name.lower()
            cls.registry[interface_id] = cls
            
        super(DBInterfaceMeta, cls).__init__(name, bases, dct)
        
class DBInterface(object):
    __metaclass__ = DBInterfaceMeta
    
pp.pprint(DBInterface.registry)
    # has nothing

class FirstInterface(DBInterface):
    pass

print
pp.pprint(DBInterface.registry)
    # has one
    
class SecondInterface(DBInterface):
    pass

class SecondInterfaceModified(SecondInterface):
    pass

print
pp.pprint(DBInterface.registry)
    # has three
    

{ }

{ 'firstinterface': <class '__main__.FirstInterface'>}

{ 'firstinterface': <class '__main__.FirstInterface'>,
  'secondinterface': <class '__main__.SecondInterface'>,
  'secondinterfacemodified': <class '__main__.SecondInterfaceModified'>}


In [25]:
SHOULD_WE_USE_META_CLASSES = """

    When Should You Use Metaclasses?
    
        Metaclasses are deeper magic than 99% of users should ever worry about. 
        
        If you wonder whether you need them, you don’t 
        
        (the people who actually need them know with certainty 
        that they need them, and don’t need an explanation about why).
        
        – Tim Peters
"""

print(SHOULD_WE_USE_META_CLASSES)




    When Should You Use Metaclasses?
    
        Metaclasses are deeper magic than 99% of users should ever worry about. 
        
        If you wonder whether you need them, you don’t 
        
        (the people who actually need them know with certainty 
        that they need them, and don’t need an explanation about why).
        
        – Tim Peters



### \_\_mro\_\_ and mro()

```
class.__mro__

    This attribute is a tuple of classes that are considered when 
    looking for base classes during method resolution.

class.mro()
    
    This method can be overridden by a metaclass to customize the 
    method resolution order for its instances. 
    
    It is called at class instantiation, and 
    
    its result is stored in __mro__.    
```

- [**Method Resolution Order**](http://python-history.blogspot.com/2010/06/method-resolution-order.html)
   
   by  Guido van Rossum


- [**What does *mro()* do in Python?**](http://stackoverflow.com/questions/2010692/what-does-mro-do-in-python)


- [**class.\_\_mro\_\_**](https://docs.python.org/2/library/stdtypes.html#class.__mro__)

**mro()** stands for *Method Resolution Order*. 

It returns a list of types the class is derived from, in the order they are searched for methods.

In [29]:
"""
Example 1
"""

class A(object): pass
class B(A): pass
class C(A): pass
class D(C): pass
class E(B,C): pass

print A.__mro__ # A, object
print B.__mro__ # B, A, object
print C.__mro__ # C, A, object
print D.__mro__ # D, C, A, object
print E.__mro__ # E, B, C, A, object

print

print A.mro() # A, object
print B.mro() # B, A, object
print C.mro() # C, A, object
print D.mro() # D, C, A, object
print E.mro() # E, B, C, A, object


(<class '__main__.A'>, <type 'object'>)
(<class '__main__.B'>, <class '__main__.A'>, <type 'object'>)
(<class '__main__.C'>, <class '__main__.A'>, <type 'object'>)
(<class '__main__.D'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>)
(<class '__main__.E'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>)

[<class '__main__.A'>, <type 'object'>]
[<class '__main__.B'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.C'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.D'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.E'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>]


In [38]:
"""
MRO Algorithms in Python

    http://python-history.blogspot.com/2010/06/method-resolution-order.html

    In languages that use multiple inheritance, the order in which base classes are searched when looking for a method 
    is often called the Method Resolution Order, or MRO. (In Python this also applies to other attributes.) 
    
    For languages that support single inheritance only, the MRO is uninteresting; but when multiple inheritance comes 
    into play, the choice of an MRO algorithm can be remarkably subtle. Python has known at least three different 
    MRO algorithms: 
    
        classic, 
        Python 2.2 new-style, and 
        Python 2.3 new-style (a.k.a. C3). 
        
    Only the latter survives in Python 3.

    Classic classes used a simple MRO scheme: 
    
        when looking up a method, base classes were searched using a simple 
        
            depth-first 
            left-to-right 
            
        scheme. 
        
        The first matching object found during this search would be returned. 
        
        For example, consider these classes:

"""

class A:
    def save(self): print 'save() in A'

class B(A): pass

class C:
    def save(self): print 'save() in C'

class D(B, C): pass

D().save() # save() in A
print


save() in A



In [39]:
"""
    Python 2.2 new-style
    
    diamond inheritance
        
    The computation of the MRO was officially documented as using a 
        depth-first left-to-right 
    traversal of the classes as before. 
    
    If any class was duplicated in this search, all but the last occurrence 
    would be deleted from the MRO list.
    
"""

class A(object):
    def save(self): print 'save() in A'

class B(A): pass

class C(A):
    def save(self): print 'save() in C'

class D(B, C): 
    """
    If this is Python 2.2:
    
        Classic would be: D, B, A, C, A, object
        Only keep last A, so become:  D, B, C, A, object
        
    """
    pass

print A.mro() # A, object
print B.mro() # B, A, object
print C.mro() # C, A, object
print D.mro() # D, B, C, A, object <== D, B, A, C, A, object
D().save() # save() in C
print


[<class '__main__.A'>, <type 'object'>]
[<class '__main__.B'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.C'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>]
save() in C



In [42]:
"""
    New style classes have a common root: object. 
    
    This make diamond inheritance happens very common.
"""
class A(object):
    def save(self): print 'save() in A'

class B(object): 
    def save(self): print 'save() in B'

class C(A,B): 
    pass

print A.mro() # A, object
print B.mro() # B, object
print 
print C.mro() # C, A, B, object
C().save() # save() in A
print

class C(B,A): 
    pass

print C.mro() # C, B, A, object
C().save() # save() in B
print

[<class '__main__.A'>, <type 'object'>]
[<class '__main__.B'>, <type 'object'>]

[<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <type 'object'>]
save() in A

[<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <type 'object'>]
save() in B



In [44]:
"""
    Python 2.2 new-style MRO approach caused inconsistency, not only in below the special case.
    
    Thus, in Python 2.3, abandoned the home-grown 2.2 MRO algorithm in favor of the academically 
    vetted C3 algorithm.
    
    C3 Linearization algorithm is about monotonicity:
    
        "A Monotonic Superclass Linearization for Dylan" (K. Barrett, et al, presented at OOPSLA'96).
        
    Basically, the idea behind C3 is that if you write down all of the ordering rules imposed by 
    inheritance relationships in a complex class hierarchy, the algorithm will determine a monotonic 
    ordering of the classes that satisfies all of them. 
    
    If such an ordering can not be determined, the algorithm will fail.
    
    The example below will cause error for class E(C,D):
    
        TypeError: Error when calling the metaclass bases
        Cannot create a consistent method resolution order (MRO) for bases A, B
"""

class A(object): pass
class B(object): pass
class C(A,B): pass
class D(B,A): pass
class E(C,D): pass


TypeError: Error when calling the metaclass bases
    Cannot create a consistent method resolution
order (MRO) for bases B, A

In [48]:
"""
    Change mro using a metaclass
    
    http://pybites.blogspot.com.au/2009/01/mro-magic.html
    
"""

class A(object):

    def a_method(self):
        print("A")

class B(object):

    def b_method(self):
        print("B")

class MROMagicMeta(type):

    def mro(cls):
        """ overide default mro() """
        return (cls, B, object)

class C(A):
    __metaclass__=MROMagicMeta
    def c_method(self):
        print("C")


mycls = C()
print mycls                 # <__main__.C object at 0x...>
mycls.c_method()
try:
    mycls.a_method()        # C
except Exception as e:
    print type(e), e        # <type 'exceptions.AttributeError'> 'C' object has no attribute 'a_method'
mycls.b_method()            # B
print type(mycls).__mro__   # (<class '__main__.C'>, <class '__main__.B'>, <type 'object'>)
print type(mycls).__bases__ # (<class '__main__.A'>,)


C
<type 'exceptions.AttributeError'> 'C' object has no attribute 'a_method'
B
(<class '__main__.C'>, <class '__main__.B'>, <type 'object'>)
(<class '__main__.A'>,)


In [56]:
"""
    A more complex example to change MRO:
    
        change at runtime
    
    Details of the hask is at:
        http://stackoverflow.com/questions/20822850/change-python-mro-at-runtime
"""

class change_mro_meta(type):
    def __new__(cls, cls_name, cls_bases, cls_dict):
        out_cls = super(change_mro_meta, cls).__new__(cls, cls_name, cls_bases, cls_dict)
        out_cls.change_mro = False
        out_cls.hack_mro   = classmethod(cls.hack_mro)
        out_cls.fix_mro    = classmethod(cls.fix_mro)
        out_cls.recalc_mro = classmethod(cls.recalc_mro)
        return out_cls

    @staticmethod
    def hack_mro(cls):
        cls.change_mro = True
        cls.recalc_mro()

    @staticmethod
    def fix_mro(cls):
        cls.change_mro = False
        cls.recalc_mro()

    @staticmethod
    def recalc_mro(cls):
        # Changing a class' base causes __mro__ recalculation
        cls.__bases__  = cls.__bases__ + tuple()

    def mro(cls):
        default_mro = super(change_mro_meta, cls).mro()
        if hasattr(cls, "change_mro") and cls.change_mro:
            print "change_mro"
            return default_mro[1:2] + default_mro
        else:
            return default_mro

class A(object):
    def __init__(self):
        print "__init__ A"
        self.hello()

    def hello(self):
        print "A hello"

class B(A):
    __metaclass__ = change_mro_meta
    def __init__(self):
        self.hack_mro()
        super(B, self).__init__()
        self.fix_mro()
        print "__init__ B"
        self.msg_str = "B"
        self.hello()

    def hello(self):
        print "%s hello" % self.msg_str

a = A()
print type(a).mro(), type(a).__bases__
print
b = B()
print type(b).mro(), type(b).__bases__

__init__ A
A hello
[<class '__main__.A'>, <type 'object'>] (<type 'object'>,)

change_mro
__init__ A
A hello
__init__ B
B hello
[<class '__main__.B'>, <class '__main__.A'>, <type 'object'>] (<class '__main__.A'>,)


### [PEP 3119 -- Introducing Abstract Base Classes](https://www.python.org/dev/peps/pep-3119/)

**Abstract**

This is a proposal to add Abstract Base Class (ABC) support to Python 3000. It proposes:

1. A way to overload isinstance() and issubclass() .
2. A new module abc which serves as an "ABC support framework". 
    It defines a metaclass for use with ABCs and a decorator that can be used to define abstract methods.
3. Specific ABCs for containers and iterators, to be added to the collections module.

Much of the thinking that went into the proposal is not about the specific mechanism of ABCs, as contrasted with Interfaces or Generic Functions (GFs), but about clarifying philosophical issues like 

    "what makes a set", 
    "what makes a mapping" and 
    "what makes a sequence".

There's also a companion PEP 3141 , which defines ABCs for numeric types.

**Dilemma** is exist in choosing between standardizing more, fine-grained ABCs or fewer, course-grained ones.

###### Example 1: complex numbers

    At one stage, PEP 3141 introduced the following stack of base classes used for complex numbers: 
        MonoidUnderPlus, AdditiveGroup, Ring, Field, Complex (each derived from the previous). 
        
    And several other algebraic categorizations were left out: 
        Algebraic, Transcendental, and IntegralDomain, and PrincipalIdealDomain.
        
###### Example 2: sequence types

    Another example would be someone who wants to define a generic function ( PEP 3124 ) for any sequence 
    that has an append() method. 
    
    The Sequence ABC (see below) doesn't promise the append() method, while MutableSequence requires NOT 
    only append() but also various other mutating methods.
    
##### Proposed solution:

    A metaclass for use with ABCs that will allow us to add an ABC as a "virtual base class" 
    (not the same concept as in C++) to any class, including to another ABC. 
    
    This allows the standard library to define ABCs Sequence and MutableSequence and register 
    these as virtual base classes for built-in types like basestring , tuple and list , so that 
    for example the following conditions are all true:
    

In [3]:

from collections import Sequence, MutableSequence

print isinstance([], Sequence)
print issubclass(list, Sequence)
print issubclass(list, MutableSequence)
print isinstance((), Sequence)
print not issubclass(tuple, MutableSequence)
print isinstance("", Sequence)
print issubclass(MutableSequence, Sequence)

True
True
True
True
True
True
True


### [Customizing instance and subclass checks](https://docs.python.org/2/reference/datamodel.html#customizing-instance-and-subclass-checks)

New in version 2.6.

The following methods are used to override the default behavior of the isinstance() and issubclass() built-in functions.

In particular, the metaclass abc.ABCMeta implements these methods in order to allow the addition of Abstract Base Classes (ABCs) as “virtual base classes” to any class or type (including built-in types), including other ABCs.

    class.__instancecheck__(self, instance)

        Return true if instance should be considered a (direct or indirect) instance of class. 
        If defined, called to implement 
        
            isinstance(instance, class).

    class.__subclasscheck__(self, subclass)
    
        Return true if subclass should be considered a (direct or indirect) subclass of class. 
        If defined, called to implement
        
            issubclass(subclass, class).

Note that these methods are looked up on the type (metaclass) of a class. They **CANNOT** be defined as class methods in the actual class. This is consistent with the lookup of special methods that are called on instances, only in this case the instance is itself a class.

[Overloading isinstance() and issubclass()](https://www.python.org/dev/peps/pep-3119/#overloading-isinstance-and-issubclass)

    The overloading works as follows: 
    
    The call isinstance(x, C) first checks whether C.__instancecheck__ exists, and 
        if so, calls C.__instancecheck__(x) instead of its normal implementation. 
        
    Similarly, the call issubclass(D, C) first checks whether C.__subclasscheck__ exists, and 
        if so, calls C.__subclasscheck__(D) instead of its normal implementation.
        
    Note:
    
    The magic names are not __isinstance__ and __issubclass__ ; 
    this is because the REVERSAL of the arguments could cause confusion, 
        especially for the issubclass() overloader.


In [6]:
"""
Demonstrate a simple implementation
"""

class ABCMeta(type):

    def __instancecheck__(cls, inst):
        """Implement isinstance(inst, cls)."""
        return any(cls.__subclasscheck__(c)
                   for c in (type(inst), inst.__class__))

    def __subclasscheck__(cls, sub):
        """Implement issubclass(sub, cls)."""
        candidates = cls.__dict__.get("__subclass__", set()) or (cls,)
        return any(c in candidates for c in sub.mro())

class Sequence(object):
    __metaclass__=ABCMeta
    __subclass__ = (list, tuple)

class AppendableSequence(Sequence):
    __subclass__ = (list,)

print issubclass(list, Sequence)
print issubclass(tuple, Sequence)
print issubclass(list, AppendableSequence)
print isinstance([], AppendableSequence)
print not issubclass(tuple, AppendableSequence)
print not isinstance((), AppendableSequence)

True
True
True
True
True
True


### [abc – Abstract Base Classes](https://pymotw.com/2/abc/)

Why use Abstract Base Classes?

    Abstract base classes are a form of interface checking more strict than individual hasattr() checks for particular methods.
    
    By defining an abstract base class, you can define a common API for a set of subclasses. 
    
    This capability is especially useful in situations where a third-party is going to provide implementations, such as with plugins to an application, but can also aid you when working on a large team or with a large code-base where keeping all classes in your head at the same time is difficult or not possible.

How ABCs Work

    abc works by marking methods of the base class as abstract, and then registering concrete classes as implementations of the abstract base. 
    
    If your code requires a particular API, you can use issubclass() or isinstance() to check an object against the abstract class.

In [15]:
import abc

""" 
Let’s start by defining an abstract base class to represent the API of a set of plugins for saving and loading data. 
"""

class PluginBase(object):
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def load(self, input):
        """Retrieve data from the input source and return an object."""
        return
    
    @abc.abstractmethod
    def save(self, output, data):
        """Save the data object to the output."""
        return
    
"""
There are two ways to indicate that a concrete class implements an abstract: 

    register the class with the abc or 
    
    subclass directly from the abc.
    
"""


""" Registering a Concrete Class """

class RegisteredImplementation(object):
    
    def load(self, input):
        return input.read()
    
    def save(self, output, data):
        return output.write(data)

PluginBase.register(RegisteredImplementation)

print 'Subclass:', issubclass(RegisteredImplementation, PluginBase)
print 'Instance:', isinstance(RegisteredImplementation(), PluginBase)
print


""" Implementation Through Subclassing """

class SubclassImplementation(PluginBase):
    
    def load(self, input):
        return input.read()
    
    def save(self, output, data):
        return output.write(data)

print 'Subclass:', issubclass(SubclassImplementation, PluginBase)
print 'Instance:', isinstance(SubclassImplementation(), PluginBase)
print


"""
A side-effect of using direct subclassing is it is possible to find all of the implementations 
of your plugin by asking the base class for the list of known classes derived from it 
(this is not an abc feature, all classes can do this).

"""

for sc in PluginBase.__subclasses__():
    print sc.__name__
    """
    RegisteredImplementation is not among the list of subclasses 
    because it is NOT actually derived from the base.
    """

Subclass: True
Instance: True

Subclass: True
Instance: True

SubclassImplementation


##### Incomplete Implementations

    Another benefit of subclassing directly from your abstract base class is that the subclass cannot be instantiated unless it fully implements the abstract portion of the API. 

    This can keep half-baked implementations from triggering unexpected errors at runtime.

In [16]:
class IncompleteImplementation(PluginBase):
    
    def save(self, output, data):
        return output.write(data)

PluginBase.register(IncompleteImplementation)

if __name__ == '__main__':
    print 'Subclass:', issubclass(IncompleteImplementation, PluginBase)
    print 'Instance:', isinstance(IncompleteImplementation(), PluginBase)

Subclass: True
Instance:

TypeError: Can't instantiate abstract class IncompleteImplementation with abstract methods load

##### Concrete Methods in ABCs

    Although a concrete class must provide an implementation of an abstract methods, the abstract base class can also provide an implementation that can be invoked via super(). 
    
    This lets you re-use common logic by placing it in the base class, but force subclasses to provide an overriding method with (potentially) custom logic.


In [18]:
from cStringIO import StringIO

class ABCWithConcreteImplementation(object):
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def retrieve_values(self, input):
        print 'base class reading data'
        return input.read()

class ConcreteOverride(ABCWithConcreteImplementation):
    
    def retrieve_values(self, input):
        base_data = super(ConcreteOverride, self).retrieve_values(input)
        print 'subclass sorting data'
        response = sorted(base_data.splitlines())
        return response

input = StringIO("""line one
line two
line three
""")

reader = ConcreteOverride()
print reader.retrieve_values(input)
print

base class reading data
subclass sorting data
['line one', 'line three', 'line two']



##### Abstract Properties

    If your API specification includes attributes in addition to methods, you can require the attributes in concrete classes by defining them with @abstractproperty.
    

In [19]:
class Base(object):
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractproperty
    def value(self):
        return 'Should never get here'


class Implementation(Base):
    
    @property
    def value(self):
        return 'concrete property'


try:
    b = Base()
    print 'Base.value:', b.value
except Exception, err:
    print 'ERROR:', str(err)

i = Implementation()
print 'Implementation.value:', i.value

ERROR: Can't instantiate abstract class Base with abstract methods value
Implementation.value: concrete property


In [20]:
"""
You can also define abstract read/write properties.
"""

class Base(object):
    __metaclass__ = abc.ABCMeta
    
    def value_getter(self):
        return 'Should never see this'
    
    def value_setter(self, newvalue):
        return

    value = abc.abstractproperty(value_getter, value_setter)


class PartialImplementation(Base):
    
    @abc.abstractproperty
    def value(self):
        return 'Read-only'


class Implementation(Base):
    
    _value = 'Default value'
    
    def value_getter(self):
        return self._value

    def value_setter(self, newvalue):
        self._value = newvalue

    value = property(value_getter, value_setter)


try:
    b = Base()
    print 'Base.value:', b.value
except Exception, err:
    print 'ERROR:', str(err)

try:
    p = PartialImplementation()
    print 'PartialImplementation.value:', p.value
except Exception, err:
    print 'ERROR:', str(err)

i = Implementation()
print 'Implementation.value:', i.value

i.value = 'New value'
print 'Changed value:', i.value

ERROR: Can't instantiate abstract class Base with abstract methods value
ERROR: Can't instantiate abstract class PartialImplementation with abstract methods value
Implementation.value: Default value
Changed value: New value


In [21]:
"""
Below is the version using decorator
"""

class Base(object):
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractproperty
    def value(self):
        return 'Should never see this'
    
    @value.setter
    def value(self, newvalue):
        return


class Implementation(Base):
    
    _value = 'Default value'
    
    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, newvalue):
        self._value = newvalue


i = Implementation()
print 'Implementation.value:', i.value

i.value = 'New value'
print 'Changed value:', i.value

Implementation.value: Default value
Changed value: New value


### Collection Types

The collections module defines several abstract base classes related to container (and containable) types.

- General container classes:

        Container
        Sized
    
- Iterator and Sequence classes:

        Iterable
        Iterator
        Sequence
        MutableSequence

- Unique values:

        Hashable
        Set
        MutableSet

- Mappings:

        Mapping
        MutableMapping
        MappingView
        KeysView
        ItemsView
        ValuesView
    
- Miscelaneous:

        Callable
    
In addition to serving as detailed real-world examples of abstract base classes, 
Python’s built-in types are automatically registered to these classes when you 
import collections. 

This means you can safely use isinstance() to check parameters in your code to 
ensure that they support the API you need. 

The base classes can also be used to define your own collection types, since 
many of them provide concrete implementations of the internals and only need 
a few methods overridden. 

Refer to the standard library docs for collections for more details.

### [What is a metaclass in Python?](http://stackoverflow.com/questions/100003/what-is-a-metaclass-in-python)

This is a good post. Can read it for deeper understanding.
